In [1]:
# Importation des bases
import pandas as pd
import functions as fc
import numpy as np

dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

dict_keys(['rejets', 'etablissements', 'emissions', 'Trait_dechets_non_dangereux', 'Trait_dechets_dangereux', 'Prod_dechets_non_dangereux', 'Prod_dechets_dangereux', 'Prelevements'])

In [2]:
df = fc.clean_data_etab(
    dict_data['etablissements']
    )

In [3]:
# 56 % de match
request_template = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{"multi_match" : {{"query":"{nom_etablissement}",  "type":"best_fields", "fields": [ "denom", "enseigne", "nom_comm_et", "adr_et_l1","adr_et_l2", "denom_condense", "enseigne_et1" ],"tie_breaker": 0.1}}}},
        {{ "match": {{ "sir_adr_et_com_lib":  "{commune}" }}}}
      ],
      "filter": [
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''



In [7]:
df.columns

Index(['identifiant', 'nom_etablissement', 'numero_siret_true', 'adresse',
       'code_postal', 'commune', 'departement', 'region', 'coordonnees_x',
       'coordonnees_y', 'code_epsg', 'code_ape', 'libelle_ape', 'code_eprtr',
       'libelle_eprtr', 'geometry', 'x', 'y', 'code_apet', 'siret_elastic',
       'match', 'nom_etab_elastic', 'textual_distance'],
      dtype='object')

In [9]:
es = fc.elastic()
es.indices.get_mapping(index = 'sirus_2020')

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'sirus_2020_e_3_ngr_bool': {'mappings': {'properties': {'adr_depcom': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_cedex': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_compl': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_distsp': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'adr_et_l1': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256},
      'ngr': {'type': 'text', 'analyzer': 'ngram_analyzer'},
      'stem': {'type': 'text', 'analyzer': 'stemming'}},
     'copy_to': ['fourretout', 'rs_adr'],
     'norms': False},
    'adr_et_l2': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256},
      'ngr': {'type': 'text', 'analyzer': 'ngram_analyzer'}},
     'copy_to': ['fourretout', 'rs_adr'],
     'norms': False},
  

In [12]:
# 69 %
request_template = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{"multi_match" : {{"query":"{nom_etablissement}",  "type":"best_fields", "fields": [ "denom", "enseigne", "nom_comm_et", "adr_et_l1","adr_et_l2", "denom_condense", "enseigne_et1" ],"tie_breaker": 0.1}}}},
        {{ "fuzzy": {{ "sir_adr_et_com_lib":  "{commune}"}}}}
      ],
      "filter": [
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''

In [24]:
request_template = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{"multi_match" : {{"query":"{nom_etablissement}",  "type":"best_fields", "fields": [ "denom", "enseigne", "nom_comm_et", "adr_et_l1","adr_et_l2", "denom_condense", "enseigne_et1" ],"tie_breaker": 0.1}}}},
        {{ "fuzzy": {{ "sir_adr_et_com_lib":  {{"value":  "{commune}", "fuzziness": 1}}}}}}
      ],
      "filter": [
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''

In [25]:
out = fc.pipeline_request(df, request_template, cols = ['nom_etablissement', 'commune','code_apet'])

8963
Matched OK False    0.69508
True     0.30492
Name: match, dtype: float64


/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [5]:
request_template_geo = '''{{
"query": {{ "bool": {{ "should": {{ "match": {{ "rs_denom":   "{nom_etablissement}" }} }}, "filter": [ {{ "geo_distance": {{ "distance": "0.5km", "location": {{ "lat": "{y}",  "lon": "{x}" }}  }} }}]   }}  }}, "size": 1
}}
'''

In [ ]:
out[["nom_etablissement","nom_etab_elastic", "textual_distance", "match"]].head(20)

## Les émissions de CO2

On va commencer par se focaliser sur les gros émetteurs de CO2

In [ ]:
emissions = dict_data['emissions']
emissions_co2 = emissions.loc[emissions['polluant'].str.contains("CO2")]
emissions_co2 = emissions_co2.loc[emissions_co2['polluant'].str.contains('total')]
emissions_co2.head(1)

In [ ]:
emissions_co2['quantite'].div(1000).plot.hist(logx = True)

In [ ]:
dict_data.head(1)

In [ ]:
dict_data['emissions']["annee_emission"].value_counts()

In [ ]:
dict_data['emissions']["milieu"].value_counts()

In [ ]:
dechets_dang = dict_data['Trait_dechets_dangereux']

In [ ]:
dict_data['etablissements']['identifiant'].nunique()

In [ ]:
dict_data['etablissements'].head(1)

In [ ]:
dict_data